In [2]:
import pandas as pd
import os

In [1]:
#filenames = [i for i in os.listdir() if i.endswith('.csv')]
#df = pd.concat([pd.read_csv(f) for f in filenames ])

In [7]:
def price_to_int(df,col):
    df[col] = df[col].str.replace(' zł', '')
    df[col] = df[col].str.replace(' ','')
    df[col] = df[col].str.replace('-','0')
    df[col] = df[col].str.replace('None','0').astype(int)

In [8]:
df[['tytul', 'rok_powstania']] = df.tytul_rok.str.rsplit(',',1,expand=True)
df[['pozycja', 'dzien','miesiac','rok']] = df.data_aukcji.str.rsplit(' ',3,expand=True)
months = {'stycznia':1,'lutego':2,'marca':3,'kwietnia':4,'maja':5,'czerwca':6,'lipca':7,'sierpnia':8,'września':9,'października':10,'listopada':11,'grudnia':12}
df.miesiac=df['miesiac'].map(months)
df.autor = df.autor.fillna('-')
price_to_int(df, 'cena_wywolawcza')
price_to_int(df, 'cena_sprzedazy')
df['aukcja'] = pd.to_datetime(dict(year=df.rok, month=df.miesiac, day=df.dzien))
df[['rozmiar_cl', 'unit']] = df.rozmiar.str.rsplit(' ',1,expand=True)
df[['wymiar_x', 'wymiar_y']] = df.rozmiar_cl.str.rsplit(' x ',1,expand=True).apply(pd.to_numeric, errors='coerce').fillna(1.0)
df['wielkosc'] = df.wymiar_x * df.wymiar_y
df.technika = ', ' + df.technika
df[['techniki', 'material']] = df.technika.str.rsplit(', ',1,expand=True)
df.techniki = df.techniki.str[2:]
df['rok_pow_new'] = df.rok_powstania.str.extract('(\d+)')
df = df.drop(columns=['technika', 'nr_dziela', 'tytul_rok', 'pozycja', 'data_aukcji', 'dzien', 'miesiac', 'rok', 'rozmiar', 'unit', 'rozmiar_cl'])
df.head()
df.index = range(len(df.index))

,autor,status,cena_wywolawcza,cena_sprzedazy,tytul,rok_powstania,aukcja,wymiar_x,wymiar_y,wielkosc,techniki,material,rok_pow_new
0,Józef Jarema,zarchiwizowany,3500,3500,KOMPOZYCJA,1957,2020-11-03,20.0,29.0,580.0,"gwasz, kredki",karton,1957
1,Jacek Yerka,zarchiwizowany,500,4800,GĄSZCZ KALIGRAFICZNY-H,2020,2020-11-03,29.6,21.0,621.6,ołówek,karton,2020
2,Waldemar Jerzy Marszałek,dostępny w Galerii,2500,0,POSTAĆ W CZERWONEJ CZAPCE,2008,2020-11-03,36.0,34.0,1224.0,olej,płótno,2008
3,Eugeniusz Get-Stankiewicz,zarchiwizowany,800,800,OEDIPUS REX,1980,2020-11-03,43.0,30.0,1290.0,serigrafia,papier,1980
4,Franciszek Starowieyski,zarchiwizowany,4500,6700,OŚMIOKROTNY PROFIL,1999,2020-11-03,44.0,29.5,1298.0,pastel,papier,1999


In [3]:
df = pd.read_csv('agraart_cleaned.csv')
df.index = range(len(df.index))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23065 entries, 0 to 23064
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   autor            23065 non-null  object 
 1   status           23065 non-null  object 
 2   cena_wywolawcza  23065 non-null  int64  
 3   cena_sprzedazy   23065 non-null  int64  
 4   tytul            23065 non-null  object 
 5   rok_powstania    15714 non-null  object 
 6   aukcja           23065 non-null  object 
 7   wymiar_x         23065 non-null  float64
 8   wymiar_y         23065 non-null  float64
 9   wielkosc         23065 non-null  float64
 10  techniki         21121 non-null  object 
 11  material         23065 non-null  object 
 12  rok_pow_new      15544 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 2.3+ MB


In [29]:
df.loc[df['autor'].str.contains('XVII wieku Malarz'), 'autor'] = '-'
df.autor = df.autor.str.lstrip()

autors = sorted(df.autor.unique().tolist())
with open('malarze.txt', 'w') as f:
    for item in autors:
        f.write("%s\n" % item)

In [204]:
df_tmp = df[(df.rok_pow_new.isnull()) & (df.rok_powstania.notnull())]
df_tmp.rok_powstania.replace({'KONIEC XIX W.': '1895'}, regex=True)
df_tmp.iloc[50:100, [0,4,5,12]]
#df_tmp.count()

,autor,tytul,rok_powstania,rok_pow_new
12783,Czesław (I. Zygmuntowicz) Wasilewski,ZACHÓD SŁOŃCA,SARNY U WODOPOJU,NaN
12802,Christian Gotthard Hirsch,ŚNIEŻKA,LATA DWUDZIESTE XX W.,NaN
12835,-,BULIERA,KONIEC XIX W.,NaN
12841,-,SOKÓŁ,XX W.,NaN
12842,-,WENUS Z MILO,II POŁ. XIX W.,NaN
12903,Anna Ewa (Hannah) Miarczyńska,"NASIONA, Z CYKLU &#8222;ANIOŁY OPIEKUŃCZE&#8221;",,NaN
12979,Jan Lebenstein,ILUSTRACJI DO &#8222;FOLWARKU ZWIERZĘCEGO&#8220;,,NaN
12982,Ryszard Łagodziec,"ROŚLINY WIOSNĄ W OGRÓDKU, Z CYKLU &#8222;PEJZAŻ""",,NaN
13011,Karol Ferster (Charlie),&#8222;MARYNARZU STRZEŻ SIĘ&#8221; (PROD. ANG.),,NaN
13041,-,KOSZ,KONIEC XIX W.,NaN


In [205]:
#df.to_csv( "agraart_cleaned.csv", index=False, encoding='utf-8-sig')